<a href="https://colab.research.google.com/github/sayantan-2/comfyui_colab_Flux/blob/main/comfyui_colab_Flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the [ComfyUI](https://github.com/comfyanonymous/ComfyUI) repo and install the requirements.

In [ ]:
!git clone https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI
%pip install -q xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117


Git clone the [ComfyUI-GGUF](https://github.com/city96/ComfyUI-GGUF) repo and follow the instructions.

In [ ]:
%cd custom_nodes
%pip install --upgrade gguf
!rm -rf ComfyUI-GGUF
!git clone https://github.com/city96/ComfyUI-GGUF
%cd ..

Download the vae and dual text encoder needed for FLUX

In [ ]:
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P ./models/text_encoders/
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P ./models/text_encoders/
!wget -q --show-progress https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors -P ./models/vae/

Download the **FLUX.1-dev-gguf** models.  
The **FP8 version** runs smoothly on the free tier (without LoRA), but if you want to use it with a LoRA, it requires more resources than the free Colab tier provides.  
In that case, you can opt for the gguf quantized versions instead. For more quantized models, check [this page](https://huggingface.co/collections/QuantStack/flux-ggufs-68264cfc663d50c418940b30).


In [ ]:
# Download any one

# fp8 - needs more than free tier
# !wget -O ./models/checkpoints/flux1-dev-fp8.safetensors https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors

# flux1-dev
!wget -q --show-progress https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q6_K.gguf -P ./models/unet

# flux1-kontext-dev
# !wget -q --show-progress https://huggingface.co/QuantStack/FLUX.1-Kontext-dev-GGUF/resolve/main/flux1-kontext-dev-Q6_K.gguf -P ./models/unet

# flux1-krea-dev
# !wget -q --show-progress https://huggingface.co/QuantStack/FLUX.1-Krea-dev-GGUF/resolve/main/flux1-krea-dev-Q6_K.gguf -P ./models/unet

# flux1-schnell
# !wget -q --show-progress https://huggingface.co/city96/FLUX.1-schnell-gguf/resolve/main/flux1-schnell-Q6_K.gguf -P ./models/unet

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request
import sys

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\nComfyUI finished loading, trying to launch localtunnel...\n")
    print("The password/endpoint IP for localtunnel is:",
          urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

    p = subprocess.Popen(["lt", "--port", str(port)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

    for line in iter(p.stdout.readline, ''):
        sys.stdout.write(line)
        sys.stdout.flush()

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
!python main.py --dont-print-server